## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-26-20-48-00 +0000,wsj,Silver’s Runaway Rally Sweeps Up Amateur Inves...,https://www.wsj.com/finance/commodities-future...
1,2025-12-26-20-39-25 +0000,nyt,"Israel Recognizes Somaliland, Drawing Internat...",https://www.nytimes.com/2025/12/26/world/middl...
2,2025-12-26-20-39-14 +0000,nypost,Mets star Francisco Lindor’s podcaster wife is...,https://nypost.com/2025/12/26/us-news/mets-sta...
3,2025-12-26-20-26-00 +0000,wsj,Oil Loses Ground On Russia-Ukraine Peace Efforts,https://www.wsj.com/finance/commodities-future...
4,2025-12-26-20-25-56 +0000,nypost,Minnesota Lt. Gov. Peggy Flanagan wears hijab ...,https://nypost.com/2025/12/26/us-news/minnesot...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2365/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
95,trump,20
268,christmas,16
217,year,11
137,new,10
153,killed,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
167,2025-12-25-23-24-19 +0000,nypost,Trump launches ‘powerful and deadly’ Christmas...,https://nypost.com/2025/12/25/us-news/trump-la...,72
60,2025-12-26-17-29-14 +0000,nypost,Trump on Nigeria strikes: ISIS ‘got a very bad...,https://nypost.com/2025/12/26/us-news/trump-sp...,59
149,2025-12-26-02-59-25 +0000,nypost,Jimmy Kimmel says ‘tyranny is booming’ as he r...,https://nypost.com/2025/12/25/media/jimmy-kimm...,56
49,2025-12-26-17-57-59 +0000,nyt,Zelensky Says He’ll Meet With Trump Soon to Di...,https://www.nytimes.com/2025/12/26/world/europ...,55
77,2025-12-26-16-05-24 +0000,wapo,"Zelensky says he will meet Trump soon, citing ...",https://www.washingtonpost.com/world/2025/12/2...,53


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
167,72,2025-12-25-23-24-19 +0000,nypost,Trump launches ‘powerful and deadly’ Christmas...,https://nypost.com/2025/12/25/us-news/trump-la...
118,37,2025-12-26-11-14-54 +0000,cbc,Turkey arrests suspected ISIS member linked to...,https://www.cbc.ca/news/world/turkey-apprehend...
49,35,2025-12-26-17-57-59 +0000,nyt,Zelensky Says He’ll Meet With Trump Soon to Di...,https://www.nytimes.com/2025/12/26/world/europ...
33,34,2025-12-26-19-13-16 +0000,nypost,Heartbreaking note reveals troubled past of dr...,https://nypost.com/2025/12/26/us-news/heartbre...
90,27,2025-12-26-15-03-02 +0000,nypost,Ex-Malaysian PM Najib Razak sentenced to 15 ye...,https://nypost.com/2025/12/26/business/ex-mala...
163,26,2025-12-25-23-59-19 +0000,latimes,Federal judge blocks ICE from arresting immigr...,https://www.latimes.com/california/story/2025-...
126,22,2025-12-26-10-30-00 +0000,wsj,Tens of thousands of imports are being stopped...,https://www.wsj.com/economy/trade/customs-crac...
127,21,2025-12-26-10-02-10 +0000,nyt,Data Center Surge Reaches India as American Te...,https://www.nytimes.com/2025/12/26/business/in...
143,21,2025-12-26-04-24-29 +0000,nyt,"Peng Peiyun, 95, Dies; Official Renounced Chin...",https://www.nytimes.com/2025/12/24/world/asia/...
161,21,2025-12-26-00-51-00 +0000,latimes,She goes to police calls in a Prius. It's part...,https://www.latimes.com/california/story/2025-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
